In [ ]:
import pandas as pd
import os
import openai
from openai import OpenAI


In [ ]:
path = os.getenv('FILE_PATH')
os.chdir(path) 
# Set your OpenAI API key (replace with your actual key)
openai.api_key = os.getenv('OPENAI_API_KEY')
# Initialize the OpenAI client
client = OpenAI(api_key=openai.api_key)

In [ ]:
def analyze_alignment(capability, assessment, criteria):
    # Create a prompt for the model
    prompt1 = f"""You are a trained analyst who is an expert in identifying opportunities and recommendations given a base assessment and a set of next level criteria. Based on the Level 1 Assessment, {assessment} and subsequent Level 2 Criteria, {criteria}, identify 3 key opportunities for improvement, recommendations to realize the opportunities and the benefits of implementing the recommendations. For each the 3 identified Opportunities, the output should be basic text that includes only the following: Capability, Opportunity, Recommendation, Benefit. No numbering or other narrative or explanation is needed in the output. Please follow these instructions explicitly  """
    
# Call the OpenAI API to generate a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a good assistant"},
            {"role": "user", "content": prompt1},
        ],
        max_tokens=500,
        temperature=0
    )
    #print(prompt)
    #print(response.choices[0].message)
    alignment_strength = response.choices[0].message.content.strip().lower()
    return alignment_strength

In [ ]:
# Load your spreadsheet data into a pandas DataFrame
df = pd.read_excel('recommendation_input.xlsx')
df['Capability'] = df['Capability']
df['Assessment'] = df['Assessment']
df['Criteria'] = df['Criteria']
#df['Benefit'] = ''
#print(df)

In [ ]:
#Iterate over each Criteria in the DataFrame and perform analysis
results = []
for _, row in df.iterrows():
    capability = row['Capability']
    assessment = row["Assessment"]
    criteria = row['Criteria']
    recommend = analyze_alignment(capability, assessment, criteria)

    recommendations = recommend.split("\n\n")

    for recommendation in recommendations:
        # Replace labels and split parts by '\n'
        print(recommendation)
        parts = recommendation.replace(
            'capability:', '').replace(
            'opportunity:', '').replace(
            'recommendation:', '').replace(
            'benefit:', '').split(
            '\n')

        # Remove leading/trailing spaces
        parts = [part.strip() for part in parts if part]  # also ignore empty parts
        #print(parts)

        if len(parts) >= 3:
            opportunity, rec, benefit = parts[:3]  # assigning split parts to opportunity, rec, and benefit
            results.append([capability, opportunity, rec, benefit])

In [ ]:
print(results)

In [ ]:
output_df = pd.DataFrame(results, columns=['Capability', 'Opportunity', 'Recommendation', 'Benefit'])
output_df.to_csv('recommendations.csv', index=False)
#output_df.to_csv('recommendations.csv', index=False)
#domain_summary = pd.DataFrame({"Assessment":[assessment], "Score": [average_similarity]})
#domain_summary.to_csv('domain_summary.csv', index=False)